# 51 Peg - advanced

This section is a hands-on tutorial on how to make a run taking advantage of `EMPEROR`'s options.
Similarly to the previous tutorial, we use the 51 Peg RV data available on [GitHub](https://github.com/ReddTea/astroemperor/tree/main/tests/datafiles/51Peg/RV).

First, we start our simulation:

In [1]:
import astroemperor as emp
import numpy as np
np.random.seed(1234)


sim = emp.Simulation()
sim.load_data('51Peg')  # folder read from /datafiles/

I couldnt grab the terminal size. Trying with pandas...
Terminal size with pandas successful!
                                                                                
                   ~~ Simulation Successfully Initialized ~~                    
                                                                                


                         
Reading data from 51peg.vels                          








## General Options
We take a deeper look into the Emperor options, starting with parallelisation, and sampler backend.

### Parallelisation
Parallelisation can be done with several different libraries. The available options are:

```multiprocess_method```: To change the parallelisation scheme.

```cores__```: Misnomer for how many threads (and not cores) to use.


| Code | Library         | Pool           |
|------|-----------------|----------------|
| 0    | None            | None           |
| 1    | multiprocessing | Pool           |
| 2    | multiprocess    | Pool           |
| 3    | multiprocessing | ThreadPool     |
| 4    | pathos          | ProcessingPool |
| 5    | schwimmbad      | SerialPool     |
| 6    | schwimmbad      | JoblibPool     |
| 7    | schwimmbad      | MultiPool      |

### Sampler backend
`reddemcee` has two different backends. `PTBackend` is the default, working exclusively with RAM, and `HDFBackend`, stores the chain in an HDF5 file using `h5py`, saving it there step-by-step.

The first one is faster, but requires a higher RAM usage.

The second one is safer and less memory-hungry. To use the h5 backend, simply change the `backend_bool` attribute to `True`.


In [2]:
sim.multiprocess_method = 1  # multiprocessing Pool
sim.cores__ = 12  # threads for the run
sim.backend_bool = False  # True for h5py backend

## Engine configuration

We add ```set_engine``` with some custom options. We will use a different starting temperature ladder. A good initial ladder increases the efficiency of the adaptation, in the same way a good initial guess for the parameters increases the convergence time.

We will use a different adaptation algorithm, based on the specific heat of the system. We also change the adaptation rate, and decay timescale.

In [3]:
ntemps, nwalkers, nsweeps, nsteps = 10, 256, 2048, 1
sim.set_engine('reddemcee')


sim.engine_config['setup'] = [ntemps, nwalkers, nsweeps, nsteps]

sim.engine_config['betas'] = list(np.linspace(1, 0, ntemps))

sim.engine_config['tsw_history'] = True  # save temperature swaps per sweep
sim.engine_config['smd_history'] = True  # save swap mean distance per sweep

sim.engine_config['adapt_tau'] = 100
sim.engine_config['adapt_nu'] = 1.5
sim.engine_config['adapt_mode'] = 2  # Specific Heat

## Run Configuration
We will set up a 10\% burn-in phase.

In [4]:
sim.run_config['burnin'] = 0.1

### Model Comparison
In our example, instead of comparing BIC we will compare Evidences directly.
We can change the evidence estimation method between Curvature-aware Thermodynamic Integration (TI+), Geometric-Bridge Stepping Stones (SS+), and a Hybrid algorithm (for more details see the [reddemcee paper](https://arxiv.org/abs/2509.24870)).

In [5]:
sim.set_comparison_criteria('Evidence')
sim.set_tolerance(10)  # difference between models

sim.evidence_method = 'ss'  # stepping stones

Comparison Criteria set to Evidence
Comparison Criteria Tolerance set to 10


## Model Configuration

We feed the name of the instrument (optional), as well as the stellar mass for calculating the minimum-mass and semi-major axis, and stellar mass error (optional). We will use the Keplerian parameterisation $(P, K, \\(\phi\\), \\(e\\), \\\omega\\). We add some boundaries to speed up the process, and add some initial positions:

In [6]:
sim.instrument_names_RV = ['LICK']
sim.starmass = 1.12
sim.starmass_err = 0.04
sim.keplerian_parameterisation = 0


sim.add_condition(['Period 1', 'limits', [3, 5]])
sim.add_condition(['Amplitude 1', 'limits', [45, 60]])
sim.add_condition(['Eccentricity 1', 'limits', [0, 0.5]])

sim.add_condition(['Offset 1', 'limits', [-10., 10.]])

sim.add_condition(['Period 1', 'init_pos', [4.1, 4.3]])
sim.add_condition(['Amplitude 1', 'init_pos', [50, 60]])
sim.add_condition(['Eccentricity 1', 'init_pos', [0, 0.1]])

### Plotting Options
We add some plotting options to speed up this test a little. We will only plot the posteriors for the cold chain, and two intermediate chains. Also, we won't use the `arviz` optional plots.

In [7]:
sim.plot_posteriors['temps'] = [0]
sim.plot_trace['plot'] = False
sim.plot_gaussian_mixtures['plot'] = False

Finally, we run our simulation (it will take some minutes):

In [8]:
sim.autorun(0, 1)

                              Offset block added, OffsetBlock


                              Jitter block added, JitterBlock



Condition applied: Parameter Offset 1 attribute limits set to [-10.0, 10.0]


                                ~~ Setup Info ~~                                


Current Engine is            reddemcee 1.0
Number of cores is           12
Save location is             datalogs/51Peg/run_15/k0
Dynamical Criteria is        None
Posterior fit method is      Gaussian Mixtures
Limits constrain method is   range
Model Selection method is    Evidence


                           ~~ Automatically Saving ~~                           


Logger       : ✔
Samples      : ✘
Posteriors   : ✔
Likelihoods  : ✔
Plots: Posteriors           : ✔
Plots: Keplerian Model      : ✔
Plots: Gaussian Mixture     : ✘
Plots: Parameter Histograms : ✔
Plots: Corner               : ✔


                               ~~ Pre-Run Info ~~                               


Parameter    Prior           

100%|██████████| 20480/20480 [02:56<00:00, 115.82it/s]


temp_script.py took 176.974 seconds
Autocorrelation tolerance=50 fails. Setting to 0.
                         Calculating Gaussian Mixtures                          


100%|██████████| 2/2 [00:01<00:00,  1.40it/s]



                                 ~~ Best Fit ~~                                 


Parameter      Value (max)  Range (-+ sig)    Prior             Limits
-----------  -------------  ----------------  ----------------  -----------
Offset 1            -0.147  [-1.073  1.069]   ~𝓤 (-10.0, 10.0)  [-10.  10.]

-----------  -------------  ----------------  ---------  ---------------
Jitter 1            36.064  [-0.637  0.692]   ~𝓝 (5, 5)  [ 0.    75.852]




                                 ~~ Run Info ~~                                 






Info                                 Value
-----------------------------------  --------------------------------------------------------------------------------
Star Name                      :     51Peg
The sample sizes are           :     [472064, 472064, 472064, 472064, 472064, 472064, 472064, 472064, 472064, 472064]
Temps, Walkers, Sweeps, Steps  :     [10, 256, 2048, 1]
Model used is                  :     ['OffsetBlock', 'JitterBlock'](2)
N dat

...



                             Plotting Beta Density                              


  0%|          | 0/1 [00:00<?, ?it/s]/IronCrane/reddtea/pip_packages/astroemperor/src/astroemperor/canvas.py:2112: RuntimeWarning: divide by zero encountered in log
  y = -1/np.diff(np.log(betas))
100%|██████████| 1/1 [00:00<00:00,  4.84it/s]




                           Plotting Temperature Rates                           


100%|██████████| 9/9 [00:00<00:00,  9.94it/s]





Time Table
Time RUN                   : 00:03:03
Time POSTPROCESS           : 00:00:11
Time CALCULATE GM          : 00:00:07
Time plot_posteriors       : 00:00:11
Time plot_histograms       : 00:00:19
Time plot_keplerian_model  : 00:00:00
Time plot_betas            : 00:00:00
Time plot_beta_density     : 00:00:00
Time plot_rates            : 00:00:00
Time plot_trace            : 00:00:00

present Evidence - past Evidence > 5

 Evidence condition met!!
-1006.789 - -1330.446 > 10


                                                                                
                              ~~ End of the Run ~~                              
                                                                                


